# Why is Julia fast and can we retrofit to Python? 

## Type Inference

To generate fast code for a function f(x,y), the compiler needs to be able to infer the types of variables in f,
map them to hardware types (registers) where possible, and call specialized code paths for those types.

At compile-time, the compiler generally only knows types of x,y, but not the values, and it needs to be able
to cascade this information to infer types throughout f and in any functions called by f.

Julia and its standard library are designed so type inference is possible for code following straightforward rules.

Sometimes this requires subtle choices that would be painful to retrofit onto an existing language.



### _The return type of a function should ONLY depend on the types of its arguments_

### This needs to be run in Python

```python
# Import NUMPY for square roots

import numpy as np

# Look at type (in)stability

type(2 ** 3)
type(2 ** -3)

# How this is correct in Python but not in MATLAB

np.sqrt(1)
np.sqrt(-1)
np.sqrt(-1 + 0j)

```

### Another form of instability is the switch from _integers_ to _bigints_

```python
def fac(n):
 if n < 2:
   return 1
 else:
   return n*fac(n-1)
   
fac(20)
fac(21)

```

NB: This code is on the _tut3_ workbook

### Let's see how Julia deals with this

In [1]:
typeof(2^3)

Int64

In [2]:
typeof(2^-3)

LoadError: DomainError
while loading In[2], in expression starting on line 1

In [3]:
sqrt(1)

1.0

In [4]:
sqrt(-1)

LoadError: DomainError
while loading In[4], in expression starting on line 1

In [5]:
sqrt(-1 + 0im)

0.0 + 1.0im

### and with the factorial

In [6]:
fac(n) = (n < 2) ? 1 : n*fac(n-1);
fac(20)

2432902008176640000

In [7]:
fac(21)

-4249290049419214848

### but

In [8]:
fac(big(21))

51090942171709440000

In [9]:
factorial(21)   # in the Base.combinatorics module

LoadError: OverflowError()
while loading In[9], in expression starting on line 1

- Not all mathematical operations on numbers in Julia produce the same _type_ of result.
- But Julia is consistant or an exception is raised.
- The outcome(s) are not without being the subject of a _great_ deal of debate.

In [10]:
2/4

0.5

In [11]:
4/2

2.0

In [12]:
4 ÷ 2            # x ÷ y    =>    div(x,y)  or  x div y

2

In [13]:
4//11 / 2//7     # divide 2 rationals

14//11

In [14]:
typeof(4//11 / 2//7)

Rational{Int64} (constructor with 1 method)

In [15]:
typeof(4//11 ÷ 2//7)

Int64

In [ ]:
4//11 ÷ 2//7

In [ ]:
typeof(4.0 ÷ 3.0)

---

### Delegation O-O paradigm, as contrasted with inheritence or association

We will look at how this works using one of the original Julia examples: Modular Integers

In [ ]:
# We need to be able to import the module by munging the LOAD PATH

In [1]:
cd("/Users/malcolm/notebooks.git/PYD/code");  # This is dependent on YOUR directory structure

In [2]:
push!(LOAD_PATH,".")

3-element Array{Union(ASCIIString,UTF8String),1}:
 "/Applications/Julia-0.3.11.app/Contents/Resources/julia/local/share/julia/site/v0.3"
 "/Applications/Julia-0.3.11.app/Contents/Resources/julia/share/julia/site/v0.3"      
 "."                                                                                  

In [19]:
import ModInts;    # "import" does not add functions to the MAIN class  
ms = ModInts       # so we need to assign an identifier, in a similar fash to Python's "as" statement

ModInts

In [20]:
; cat modints.jl

# This file is a part of Julia. License is MIT: http://julialang.org/license

module ModInts
export ModInt

immutable ModInt{n} <: Integer
    k::Int
    ModInt(k) = new(mod(k,n))
end

import Base.+, Base.-, Base.*

-{n}(a::ModInt{n}) = ModInt{n}(-a.k)
+{n}(a::ModInt{n}, b::ModInt{n}) = ModInt{n}(a.k+b.k)
-{n}(a::ModInt{n}, b::ModInt{n}) = ModInt{n}(a.k-b.k)
*{n}(a::ModInt{n}, b::ModInt{n}) = ModInt{n}(a.k*b.k)

Base.convert{n}(::Type{ModInt{n}}, i::Int) = ModInt{n}(i)
Base.promote_rule{n}(::Type{ModInt{n}}, ::Type{Int}) = ModInt{n}

Base.show{n}(io::IO, k::ModInt{n}) = print(io, "$(k.k) mod $n")
Base.showcompact(io::IO, k::ModInt) = print(io, k.k)

Base.inv{n}(a::ModInt{n}) = ModInt{n}(invmod(a.k, n))

end # module


In [21]:
m1 = ms.ModInt{13}(8);
m2 = ms.ModInt{13}(4);
m3 = ms.ModInt{13}(3);

In [22]:
m1 + m2*m3

7 mod 13

In [23]:
inv(m3)

9 mod 13

In [24]:
m1 + 1

9 mod 13

In [25]:
m1 + 1//2

LoadError: < not defined for ModInt{13}
while loading In[25], in expression starting on line 1

In [26]:
m1 + 2.0

LoadError: no promotion exists for ModInt{13} and Float64
while loading In[26], in expression starting on line 1

In [27]:
m1 + int(2.0)

10 mod 13

In [28]:
mm = [ms.ModInt{13}(rand(1:13)) for i = 1:100] 

100-element Array{Any,1}:
  4
  6
  6
  3
 12
  4
  5
 11
  3
  6
 10
 12
 11
  ⋮
  6
 11
  1
 10
  5
 11
  7
  1
  6
  1
 11
 11

In [29]:
ma = reshape(mm,10,10)

10x10 Array{Any,2}:
  4  10  0   5   3   7   9   7  11   1
  6  12  7   3   2   2   2  11  12  10
  6  11  2  11  11   6   8  10   7   5
  3   9  8   5  11   9  12  11   1  11
 12   8  9  11   4   5   7   8   0   7
  4   5  1   9   3   2  11   6  12   1
  5   1  2   6   3   5   9  10   5   6
 11   1  7  10   5   0  12   8   4   1
  3   9  0   8  10   0   1  11   6  11
  6  11  6   0   3  11   1   4  11  11

In [30]:
mx = [ms.ModInt{13}(rand(1:13)) for i = 1:10]

10-element Array{Any,1}:
 11
  1
  7
  9
  7
  6
 11
  2
  2
  6

In [31]:
mb = mx' .* ma .* mx 

10x10 Array{Any,2}:
  3   6   0   1  10   7  10  11   8   1
  1  12  10   1   1  12   9   9  11   8
  7  12   7   4   6   5   5  10   7   2
 11   3  10   2   4   5   5   3   5   9
  1   4  12   4   1   2   6   8   0   8
  4   4   3   5   9   7  11   7   1  10
  7  11  11   9  10   5  10  12   6   6
  8   2   7  11   5   0   4   6   3  12
  1   5   0   1  10   0   9   5  11   2
  6   1   5   0   9   6   1   9   2   6

In [32]:
mc = map(x -> x^3, mb)

10x10 Array{ModInt{13},2}:
  1   8   0   1  12   5  12   5  5   1
  1  12  12   1   1  12   1   1  5   5
  5  12   5  12   8   8   8  12  5   8
  5   1  12   8  12   8   8   1  8   1
  1  12  12  12   1   8   8   5  0   5
 12  12   1   8   1   5   5   5  1  12
  5   5   5   1  12   8  12  12  8   8
  5   8   5   5   8   0  12   8  1  12
  1   8   0   1  12   0   1   8  5   8
  8   1   8   0   1   8   1   1  8   8

In [33]:
mc[1:5,1:5]

5x5 Array{ModInt{13},2}:
 1   8   0   1  12
 1  12  12   1   1
 5  12   5  12   8
 5   1  12   8  12
 1  12  12  12   1

## Staged code generation

### Parsed code ------> AST ------> AST (typed) ------> LLVM IR ------> [Native code]

```julia
# 1 == 1 parsed as ==(1,1)

# Try some of the following:

@code_lowered 1 == 1       # or  ==(1,1)
@code_typed   1 == 1
@code_llvm    1 == 1  
@code_native  1 == 1

```

In [34]:
@code_native 1 == 1

	.section	__TEXT,__text,regular,pure_instructions
Filename: promotion.jl
Source line: 198
	push	RBP
	mov	RBP, RSP
	cmp	RDI, RSI
Source line: 198
	sete	AL
	pop	RBP
	ret


In [ ]:
@code_native 1 == 1.0

In [3]:
# By the way

1 == 1.0

true

In [4]:
1 === 1.0

false

In [35]:
# Define a fibonanicci function (recursively)

function fib(n :: Integer)
  @assert n > 0
  return (n < 3) ? 1 : fib(n-1) + fib(n-2)
end

fib(20)

6765

In [36]:
fib(-20)

LoadError: assertion failed: n > 0
while loading In[36], in expression starting on line 1

In [37]:
fib(2.3)

LoadError: `fib` has no method matching fib(::Float64)
while loading In[37], in expression starting on line 1

In [38]:
macroexpand(:(@assert n > 0))

:(if n > 0
        nothing
    else 
        Base.error("assertion failed: n > 0")
    end)

In [39]:
@code_native(fib(20))

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[35]
Source line: 4
	push	RBP
	mov	RBP, RSP
	push	R15
	push	R14
	push	RBX
	push	RAX
	mov	RBX, RDI
	test	RBX, RBX
Source line: 4
	jle	64
Source line: 5
	cmp	RBX, 2
	jle	38
	lea	RDI, QWORD PTR [RBX - 1]
	movabs	R15, 4399269856
	call	R15
	mov	R14, RAX
	add	RBX, -2
	mov	RDI, RBX
	call	R15
	add	RAX, R14
	jmpq	5
	mov	EAX, 1
	add	RSP, 8
	pop	RBX
	pop	R14
	pop	R15
	pop	RBP
	ret
Source line: 4
	movabs	RAX, 4357696704
	mov	EDI, 16
	call	RAX
	movabs	RCX, 140367349538432
	mov	QWORD PTR [RAX], RCX
	movabs	RCX, 140367486918832
	mov	QWORD PTR [RAX + 8], RCX
	movabs	RCX, 4357640544
	mov	RDI, RAX
	mov	ESI, 4
	call	RCX


In [5]:
macroexpand(:(@printf "The value of fib(%d) is %d" 20 fib(20)))

quote 
    #220#out = Base.Printf.STDOUT
    #221###x#23473 = 20
    #222###x#23474 = fib(20)
    local #227#neg, #226#pt, #225#len, #219#exp, #223#do_out, #224#args
    Base.Printf.write(#220#out,"The value of fib(")
    if Base.Printf.isfinite(#221###x#23473)
        (#223#do_out,#224#args) = Base.Printf.decode_dec(#220#out,#221###x#23473,"",0,-1,'d')
        if #223#do_out
            (#225#len,#226#pt,#227#neg) = #224#args
            #227#neg && Base.Printf.write(#220#out,'-')
            Base.Printf.write(#220#out,Base.Printf.pointer(Base.Printf.DIGITS),#226#pt)
        end
    else 
        Base.Printf.write(#220#out,begin  # printf.jl, line 143:
                if Base.Printf.isnan(#221###x#23473)
                    "NaN"
                else 
                    if (#221###x#23473 Base.Printf.< 0)
                        "-Inf"
                    else 
                        "Inf"
                    end
                end
            end)
    end
    Base.Printf.write(#22

---

### Library calls have zero-overhead

In [6]:
systime() = ccall((:time,("libc")), Int32, ())

systime (generic function with 1 method)

In [7]:
systime()

1462540994

In [8]:
@code_native systime()

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[6]
Source line: 1
	push	RBP
	mov	RBP, RSP
	movabs	RAX, 140735708865772
Source line: 1
	call	RAX
	pop	RBP
	ret
